# Zanfran reading tables

Exploro los datos que existen en cada tabla

## Cargo librerías

In [1]:
#conda install -c plotly plotly

In [2]:
#import os
import sys

from IPython.display import display, Markdown

import numpy as np
import pandas as pd
import seaborn as sns

import plotly
from plotly import express as px
from plotly import graph_objects as go
from plotly import figure_factory as ff
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings('ignore')


In [3]:
#display(plotly.__version__)

In [4]:
# Format round off to two decimal places in pandas 
pd.options.display.float_format = '{: .2f}'.format

## Cargo funciones

Desarrollo y cargo funciones a aplicar:

### Para explorar datos

In [5]:
# Definición de funciones genéricas creadas por mi

def print_columns(df):
    display(Markdown('#### List of columns:'))
    text = '| '
    for x in range(0, df.shape[1]):
        text = text + df.columns.values[x] + ' | '
    return text

def explore_var(df,var,limit=20):
    cant=df.nunique()
    df_full = df.value_counts().rename_axis('unique_values').reset_index(name='counts')
    df_full = df_full.sort_values(by=['counts'],ascending=False)
    topLimit= df_full[0:limit]
    t1='\nThere are **'+str(cant)+'** different value in the **'+str(var)+'** variable of the dataframe.'
    if cant<=1:
        t1='\nThere is **only one** different value in the **'+str(var)+'** variable of the dataframe.'
        t2 ='The value is:'  
    elif cant>=limit:
        t2='The list of the first **'+str(limit)+'**  values:'
        df_full = topLimit
    else:
        t2 ='The list of the **'+str(cant)+'**  values:'    
    display(Markdown(t1))
    display(Markdown(t2))
    return df_full

def print_last(df):
    display(df.tail(1))
    
def missing_zero_values_table(df):
    zero_val=df.isin([0]).sum(axis=0)
    mis_val=df.isnull().sum()
    l=len(df)
    mis_val_percent=100*df.isnull().sum()/l
    zero_val_percent=100*df.isin([0]).sum()/l
    mz_table=pd.concat([zero_val,zero_val_percent,mis_val,mis_val_percent],axis=1)
    mz_table=mz_table.rename(
        columns={
            0:'Zero Values', 
            1:'% of Zero Values', 
            2:'Missing Values', 
            3:'% of Missing Values'}
    )
    mz_table['Total Missing Values']=mz_table['Zero Values']+mz_table['Missing Values']
    mz_table['% Total Missing Values']=100*mz_table['Total Missing Values']/l
    mz_table=mz_table[mz_table.iloc[:,4] != 0].sort_values(
    '% Total Missing Values', ascending=False).round(1)
    tc=str(df.shape[1])
    tr=str(df.shape[0])
    tcm=str(mz_table.shape[0])
    txt='Your selected dataframe has '+str(tc)+' columns and '+str(tr)+' Rows.'
    txt+='\nThere are '+str(tcm)+' columns that have missing values.'
    display(mz_table)
    return txt

### Para graficar

In [6]:
# Definición de funciones para graficar con Plotly creadas por mi

In [7]:
def graf_corr(df,chart=True):
    corr = df.corr()
    if chart:
        fig = ff.create_annotated_heatmap(
            z=corr.values,
            x=list(corr.columns),
            y=list(corr.index),
            annotation_text=corr.round(2).values,
            showscale=True
        )
        fig.update_layout(
            title='Correlation between variables',
            dragmode='select',
            height=750,
            hovermode='closest',
        )
        fig.show()
    display(Markdown('#### Correlation Table'))
    display(corr)

def graf_table(df,title='',height='',height_constant=20):
    fig = ff.create_table(df, height_constant=height_constant)
    fig.layout.margin.update({'t':75, 'l':50})
    fig.layout.update({'title': title})
    fig.layout.update({'height':height})
    fig.show()
    
    
def quick_view(df,desc=True,rows=True,cols=True,corr=2,zeros=True):
    print()
    if rows: 
        display(Markdown('### **Some rows** of the dataframe'))
        display(df)
        print()
    if cols:
        display(Markdown('### **Columns** of the dataframe'))
        print(print_columns(df))
        print()
    if desc:
        display(Markdown('### Dataframe **describe**'))
        display(df.describe(include='all'))
        print()
    if corr==1:
        display(Markdown('### Dataframe **correlation**'))
        graf_corr(df,False)
        print()
    elif corr==2:
        display(Markdown('### Dataframe **correlation**'))
        graf_corr(df,True)
        print()          
    if zeros:
        display(Markdown('### Dataframe **missing values**'))
        print(missing_zero_values_table(df))
        print()  

## Lectura de Tablas

In [8]:
folder = 'datasets/original/'

### account_account

In [9]:
account_account = pd.read_csv(folder+'account_account.csv')
account_account

,id,create_uid,code,create_date,reconcile,deprecated,company_id,write_uid,currency_id,note,internal_type,write_date,user_type_id,last_time_entries_checked,name,wtaxf
0,13,1,1.1.2.01.001,3/6/18 14:09,True,False,1,13,nan,nan,receivable,12/14/20 13:19,1,10/30/2019 18:53,Deudores por Ventas,NaN
1,14,1,1.1.1.03.000,3/6/18 14:09,True,False,1,1,nan,nan,other,3/15/18 17:31,5,NaN,Deudores Morosos,NaN
2,15,1,1.1.2.01.003,3/6/18 14:09,True,False,1,1,nan,nan,other,3/15/18 17:31,5,NaN,Deudores en Gestión Judicial,NaN
3,21,1,1.1.3.01.000,3/6/18 14:09,False,False,1,1,nan,nan,other,3/7/18 21:20,5,NaN,IVA Crédito Fiscal,NaN
4,23,1,1.1.3.02.200,3/6/18 14:09,False,False,1,1,nan,nan,other,3/8/18 20:08,5,NaN,Percepciones IIBB PBA,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,673,6,4.1.1.01.007,1/15/21 12:39,False,False,1,6,nan,nan,other,1/15/21 12:39,14,NaN,Ventas Cereales,NaN
214,674,13,2.1.6.09.000,1/22/21 22:06,False,False,1,13,nan,nan,other,1/22/21 22:07,10,NaN,Préstamo Sigma,cashin_out
215,675,6,2.1.1.01.009,1/25/21 20:04,False,False,1,6,nan,nan,other,1/25/21 20:04,9,NaN,Tarjeta Visa Galicia a Pagar,cashin_out
216,677,6,1.1.1.02.016,2/18/21 14:05,False,False,1,6,nan,nan,liquidity,2/18/21 14:05,3,NaN,Banco Patagonia,NaN


### account_invoice_line

In [10]:
account_invoice_line = pd.read_csv(folder+'account_invoice_line.csv')
account_invoice_line

,id,origin,create_date,price_unit,price_subtotal,write_uid,currency_id,uom_id,partner_id,create_uid,...,discount,write_date,price_subtotal_signed,name,product_id,invoice_id,quantity,layout_category_sequence,layout_category_id,purchase_line_id
0,1,NaN,7/10/18 13:22,5000.00,5000.00,6,20,23.00,4607,6,...,0.00,7/10/18 13:22,5000.00,[MKT022] Gestión Redes Sociales,2548.00,1,1.00,0.00,nan,nan
1,4,NaN,7/10/18 13:53,875.00,875.00,6,20,23.00,4608,6,...,0.00,7/10/18 13:53,875.00,[EST016] Alarma,2460.00,3,1.00,0.00,nan,nan
2,5,NaN,7/10/18 13:53,875.00,-875.00,6,20,23.00,4608,6,...,0.00,7/10/18 13:53,-875.00,[EST016] Alarma,2460.00,3,-1.00,0.00,nan,nan
3,6,NaN,7/10/18 14:09,120.00,1080.00,6,20,1.00,4611,6,...,0.00,7/13/18 19:01,1080.00,Bidones de Agua,2936.00,5,9.00,0.00,nan,nan
4,8,NaN,7/10/18 14:35,70.00,70.00,6,20,23.00,4613,6,...,0.00,7/10/18 14:35,70.00,[EST037] Honorarios Escribano,2481.00,6,1.00,0.00,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69816,85486,WOO64567,4/28/21 23:02,2654.54,2654.54,1,20,25.00,84471,1,...,0.00,4/28/21 23:02,2654.54,[BAR002x60] BARRA | Maní y cacao 60u x28g,221.00,18727,1.00,nan,nan,nan
69817,85487,WOO64567,4/28/21 23:02,0.00,0.00,1,20,1.00,84471,1,...,0.00,4/28/21 23:02,0.00,Envío gratuito,3021.00,18727,1.00,nan,nan,nan
69818,85488,WOO64568,4/28/21 23:33,2654.54,2654.54,1,20,25.00,74241,1,...,0.00,4/28/21 23:33,2654.54,[BAR006x60] BARRA | Arándanos y almendras 60u ...,3387.00,18728,1.00,nan,nan,nan
69819,85489,WOO64568,4/28/21 23:33,561.98,561.98,1,20,1.00,74241,1,...,0.00,4/28/21 23:33,561.98,[GRA008] GRANOLA | Avellanas y semillas de zap...,271.00,18728,1.00,nan,nan,nan


### account_invoice

In [11]:
account_invoice = pd.read_csv(folder+'account_invoice.csv')
account_invoice

,id,comment,date_due,create_date,partner_bank_id,number,journal_id,amount_total_company_signed,residual,partner_id,...,cbu_alias,mipymesf,mipymesf_read_only,acceptance_date,cancel_date,region_tax_control_id,date_est_payment,journal_estimated_payment,email_sent,trasnfer_opt_mipymes
0,1,nan,7/10/2018,7/10/18 13:22,nan,FCC/2018/0001,25,5000.00,0.00,4607,...,nan,NaN,NaN,nan,NaN,nan,NaN,nan,NaN,NaN
1,3,nan,7/17/2018,7/10/18 13:53,nan,FCC/2018/0002,25,0.00,0.00,4608,...,nan,NaN,NaN,nan,NaN,nan,NaN,nan,NaN,NaN
2,5,nan,7/5/2018,7/10/18 14:05,nan,FCC/2018/0003,25,1306.80,0.00,4611,...,nan,NaN,NaN,nan,NaN,nan,NaN,nan,NaN,NaN
3,6,nan,7/10/2018,7/10/18 14:35,nan,FCC/2018/0004,25,508.20,0.00,4613,...,nan,NaN,NaN,nan,NaN,nan,NaN,nan,NaN,NaN
4,7,nan,7/20/2018,7/10/18 15:06,nan,FCC/2018/0005,25,79996.74,0.00,2654,...,nan,NaN,NaN,nan,NaN,nan,NaN,nan,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15962,18724,nan,NaN,4/28/21 19:11,nan,NaN,30,3211.99,0.00,84130,...,nan,False,False,nan,NaN,nan,NaN,nan,False,NaN
15963,18725,nan,NaN,4/28/21 20:12,nan,NaN,30,3095.00,0.00,84467,...,nan,False,False,nan,NaN,nan,NaN,nan,False,NaN
15964,18726,nan,NaN,4/28/21 20:50,nan,NaN,30,3211.99,0.00,84469,...,nan,False,False,nan,NaN,nan,NaN,nan,False,NaN
15965,18727,nan,NaN,4/28/21 23:02,nan,NaN,30,3211.99,0.00,84471,...,nan,False,False,nan,NaN,nan,NaN,nan,False,NaN


### account_payment (prd2)

In [12]:
account_payment = pd.read_csv(folder+'account_payment.csv')
account_payment

,id,create_date,communication,journal_id,currency_id,partner_id,payment_method_id,payment_date,payment_difference_handling,company_id,...,withholdable_base_amount,vendorbill,automatic,manual_currency_rate_active,manual_currency_rate,manual,no_order,withholdable_invoiced_amount2,check_third_id,check_name2
0,26,7/16/18 21:22,NaN,6,20,2626.00,1,7/16/2018,open,1,...,0.00,nan,False,False,0.00,NaN,NaN,nan,nan,nan
1,1,7/10/18 13:24,NaN,9,20,4607.00,2,7/4/2018,open,1,...,0.00,nan,False,NaN,nan,NaN,NaN,nan,nan,nan
2,14,7/11/18 19:21,NaN,6,20,4640.00,2,7/11/2018,open,1,...,0.00,nan,False,False,0.00,NaN,NaN,nan,nan,nan
3,10,7/11/18 16:06,NaN,13,20,4615.00,6,7/11/2018,open,1,...,0.00,nan,False,False,0.00,NaN,NaN,nan,nan,nan
4,65,7/25/18 19:22,NaN,11,20,2611.00,4,7/25/2018,open,1,...,0.00,nan,False,False,0.00,NaN,NaN,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17785,18485,5/11/21 17:15,NaN,58,20,4810.00,4,5/11/2021,open,1,...,0.00,nan,False,True,1.00,False,False,0.00,nan,nan
17786,18484,5/11/21 17:15,NaN,58,20,4810.00,4,5/11/2021,open,1,...,0.00,nan,False,True,1.00,False,False,0.00,nan,nan
17787,18486,5/11/21 17:25,NaN,65,20,9270.00,2,5/11/2021,open,1,...,0.00,nan,False,True,1.00,False,False,0.00,nan,nan
17788,18487,5/11/21 18:25,NaN,9,20,4807.00,2,5/4/2021,open,1,...,0.00,nan,False,True,1.00,False,False,0.00,nan,nan


### account_payment (prd2)DA_Ejercitacion_22

In [13]:
# Mostrar todas las columnas
pd.set_option('display.max_columns', None)

In [14]:
account_payment2 = pd.read_csv(folder+'account_payment_22.csv')
account_payment2

,id,create_date,communication,journal_id,currency_id,partner_id,payment_method_id,payment_date,payment_difference_handling,company_id,state,writeoff_account_id,create_uid,move_name,partner_type,write_date,write_uid,name,destination_journal_id,amount,payment_type,payment_reference,payment_transaction_id,payment_token_id,receiptbook_id,document_number,readonly_amount2,inbound,outbound,check_owner_name,check_ids,checkbook_id,check_id,check_bank_id,check_name,check_issue_date,check_owner_vat,check_payment_date,check_number,payment_group_id,withholding_base_amount,withholding_number,wh_perc,customerbill,tax_withholding_id,sub_journal,withholdable_advanced_amount,withholding_non_taxable_minimum,period_withholding_amount,accumulated_amount,previous_withholding_amount,withholdable_invoiced_amount,computed_withholding_amount,total_amount,withholding_non_taxable_amount,wiz_rel,withholdable_base_amount,vendorbill,automatic,manual_currency_rate_active,manual_currency_rate,manual,no_order,withholdable_invoiced_amount2,check_third_id,check_name2
0,26,7/16/18 21:22,NaN,6,20,2626.00,1,7/16/2018,open,1,posted,nan,1,EFECT/2018/0007,customer,7/16/18 21:22,1,CUST.IN/2018/0003,nan,6873.65,inbound,nan,nan,nan,2.00,3.00,nan,True,False,Soja Alimentos S.R.L.,nan,nan,nan,nan,nan,7/16/2018,30711067759.00,NaN,0.00,24.00,0.00,NaN,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.00,nan,False,False,0.00,NaN,NaN,nan,nan,nan
1,1,7/10/18 13:24,NaN,9,20,4607.00,2,7/4/2018,open,1,posted,nan,1,BNAC$/2018/0001,supplier,7/10/18 13:25,1,SUPP.OUT/2018/0001,nan,5000.00,outbound,nan,nan,nan,1.00,3.00,nan,False,True,Agustina Sottile,nan,nan,nan,nan,nan,7/10/2018,27301317447.00,NaN,0.00,1.00,0.00,NaN,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.00,nan,False,NaN,nan,NaN,NaN,nan,nan,nan
2,14,7/11/18 19:21,NaN,6,20,4640.00,2,7/11/2018,open,1,posted,nan,1,EFECT/2018/0002,supplier,7/11/18 19:21,1,SUPP.OUT/2018/0014,nan,1200.00,outbound,nan,nan,nan,1.00,15.00,nan,False,True,Carmen,nan,nan,nan,nan,nan,7/11/2018,nan,NaN,0.00,13.00,0.00,NaN,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.00,nan,False,False,0.00,NaN,NaN,nan,nan,nan
3,10,7/11/18 16:06,NaN,13,20,4615.00,6,7/11/2018,open,1,posted,nan,1,CBBVA/2018/0006,supplier,7/11/18 16:06,1,SUPP.OUT/2018/0010,nan,28000.00,outbound,nan,nan,nan,1.00,12.00,nan,False,True,RATOWIECKI JAIME,4.00,3.00,nan,nan,4.00,7/11/2018,20042876039.00,7/11/2018,614319.00,8.00,0.00,NaN,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.00,nan,False,False,0.00,NaN,NaN,nan,nan,nan
4,65,7/25/18 19:22,NaN,11,20,2611.00,4,7/25/2018,open,1,posted,nan,1,CH3/2018/0019,customer,7/26/18 17:46,1,CUST.IN/2018/0013,nan,167172.00,inbound,nan,nan,nan,2.00,11.00,nan,True,False,Devotel Sociedad Anonima,30.00,nan,nan,64.00,nan,7/25/2018,30709456721.00,8/8/2018,57480144.00,51.00,0.00,NaN,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.00,nan,False,False,0.00,NaN,NaN,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17785,18485,5/11/21 17:15,NaN,58,20,4810.00,4,5/11/2021,open,1,draft,nan,1,ECH3/2021/0231,customer,5/11/21 17:15,1,CUST.IN/2021/1599,nan,51571.00,inbound,nan,nan,nan,2.00,8450.00,nan,True,False,Fundación Universidad de Palermo,nan,nan,nan,50.00,nan,5/11/2021,33620101449.00,5/26/2021,74125410.00,12110.00,0.00,NaN,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.00,nan,False,True,1.00,False,False,0.00,nan,nan
17786,18484,5/11/21 17:15,NaN,58,20,4810.00,4,5/11/2021,open,1,draft,nan,1,ECH3/2021/0230,customer,5/11/21 17:15,1,CUST.IN/2021/1598,nan,51569.48,inbound,nan,nan,nan,2.00,8450.00,nan,True,False,Fundación Universidad de Palermo,nan,nan,nan,50.00,nan,5/11/2021,33620101449.00,5/28/2021,74125411.00,12110.00,0.00,NaN,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,

#### Columnas

In [15]:
print(print_columns(account_payment2))

#### List of columns:

| id | create_date | communication | journal_id | currency_id | partner_id | payment_method_id | payment_date | payment_difference_handling | company_id | state | writeoff_account_id | create_uid | move_name | partner_type | write_date | write_uid | name | destination_journal_id | amount | payment_type | payment_reference | payment_transaction_id | payment_token_id | receiptbook_id | document_number | readonly_amount2 | inbound | outbound | check_owner_name | check_ids | checkbook_id | check_id | check_bank_id | check_name | check_issue_date | check_owner_vat | check_payment_date | check_number | payment_group_id | withholding_base_amount | withholding_number | wh_perc | customerbill | tax_withholding_id | sub_journal | withholdable_advanced_amount | withholding_non_taxable_minimum | period_withholding_amount | accumulated_amount | previous_withholding_amount | withholdable_invoiced_amount | computed_withholding_amount | total_amount | withholding_non_taxable_amount | wiz_rel | withhol

### product_brand

In [16]:
product_brand = pd.read_csv(folder+'product_brand.csv')
product_brand

,id,create_uid,description,write_uid,create_date,write_date,logo,partner_id,name
0,1,6,nan,6,10/13/2020 18:14,10/13/2020 18:14,nan,nan,zafrán
1,2,6,nan,6,10/13/2020 19:55,10/13/2020 19:55,nan,nan,Beepure
2,4,6,nan,6,10/13/2020 19:55,10/13/2020 19:55,nan,nan,Dell Isola
3,5,6,nan,6,10/13/2020 19:57,10/13/2020 19:57,nan,nan,Pampa Rice
4,6,6,nan,6,10/13/2020 19:57,10/13/2020 19:57,nan,nan,zafranito
5,7,6,nan,6,10/13/2020 19:58,10/13/2020 19:58,nan,nan,Cerealsol
6,8,6,nan,6,10/13/2020 19:58,10/13/2020 19:58,nan,nan,Pampa Grains
7,9,6,nan,6,10/13/2020 19:58,10/13/2020 19:58,nan,nan,La esquina de las flores
8,10,6,nan,6,10/13/2020 19:58,10/13/2020 19:58,nan,nan,Pura Frutta
9,11,6,nan,6,10/13/2020 19:59,10/13/2020 19:59,nan,nan,Pampa Vida


### product_category

In [17]:
product_category = pd.read_csv(folder+'product_category.csv')
product_category

,id,parent_left,parent_right,create_uid,name,write_uid,parent_id,write_date,create_date,type,removal_strategy_id
0,1,151,156,1,BAR,1,62.00,2/27/19 15:24,3/6/18 14:04,normal,3.00
1,4,167,168,1,MIX,1,62.00,2/27/19 15:25,3/6/18 17:24,normal,3.00
2,5,159,160,1,GAL,1,62.00,7/18/19 13:31,3/6/18 17:24,normal,3.00
3,6,149,150,1,GRAkg,1,62.00,2/27/19 15:24,3/6/18 17:24,normal,3.00
4,7,145,146,1,GRA240,1,62.00,2/27/19 15:23,3/6/18 17:25,normal,3.00
...,...,...,...,...,...,...,...,...,...,...,...
91,116,169,170,6,ING,6,62.00,5/26/20 17:03,5/26/20 17:02,normal,3.00
92,117,163,164,6,CERZafranito,6,62.00,1/15/21 12:40,1/15/21 12:38,normal,3.00
93,118,295,296,9,P3 CER,9,76.00,1/19/21 18:48,1/19/21 18:44,normal,3.00
94,119,373,374,6,PAMPAVIDA,6,120.00,4/28/21 14:15,4/28/21 14:10,normal,3.00


### product_flavor

In [18]:
product_flavor = pd.read_csv(folder+'product_flavor.csv')
product_flavor

,id,create_uid,code,create_date,name,write_uid,write_date
0,1,6,1.00,11/20/20 18:22,Combo Online,8,4/15/21 19:24
1,36,6,0.00,11/20/20 18:24,Maní y cacao,6,11/20/20 18:24
2,37,6,0.00,11/20/20 18:24,Manzana y chía,6,11/20/20 18:24
3,38,6,0.00,11/20/20 18:24,Cajú y semillas de zapallo,6,11/20/20 18:24
4,40,6,0.00,11/20/20 18:24,"Avena, coco y lino",6,11/20/20 18:24
5,41,6,0.00,11/20/20 18:24,"Algarroba, pasas y girasol",6,11/20/20 18:24
6,42,6,0.00,11/20/20 18:24,"Cacao, maní y café",6,11/20/20 18:24
7,43,6,0.00,11/20/20 18:24,"Avena, pasas y chía",6,11/20/20 18:24
8,48,6,0.00,11/20/20 18:24,"Cajú, coco y pasas",6,11/20/20 18:24
9,49,6,0.00,11/20/20 18:24,"Nueces, manzana y pasas",6,11/20/20 18:24


### product_pack_cat

In [19]:
product_pack_cat = pd.read_csv(folder+'product_pack_cat.csv')
product_pack_cat

,id,create_uid,cat_qty,name,write_uid,write_date,create_date
0,1,6,96,BARx96,6,10/14/20 13:00,10/14/20 13:00
1,2,6,60,BARx60,6,10/14/20 13:00,10/14/20 13:00
2,3,6,48,BARx48,6,10/14/20 13:00,10/14/20 13:00
3,4,6,20,GALx20,6,10/14/20 13:13,10/14/20 13:13
4,5,6,60,GALmini,6,10/14/20 13:13,10/14/20 13:13
5,6,6,7,GRAkg,6,10/14/20 13:13,10/14/20 13:13
6,7,6,24,GRA240,6,10/14/20 13:13,10/14/20 13:13
7,8,6,96,MIXx96,6,10/14/20 13:14,10/14/20 13:14
8,9,6,60,MIXx60,6,10/14/20 13:14,10/14/20 13:14
9,10,6,8,INGx8,6,10/14/20 13:14,10/14/20 13:14


### product_product

In [20]:
product_product = pd.read_csv(folder+'product_product.csv')
product_product

,id,create_date,weight,default_code,product_tmpl_id,message_last_post,create_uid,write_uid,barcode,volume,write_date,active,image_url
0,208,3/6/18 17:44,0.34,BAR002x12,209,nan,1,1,7.79819E+12,1.44,4/28/21 15:36,True,nan
1,209,3/6/18 17:44,2.69,BAR002x12x8,210,nan,1,1,1.77982E+13,12.30,4/28/21 17:45,True,nan
2,210,3/6/18 17:44,3.00,BAR004x3x32,211,nan,1,6,NaN,12.30,8/5/20 20:31,False,nan
3,211,3/6/18 17:44,0.08,BAR004x3,212,nan,1,6,7.79819E+12,0.36,1/20/21 18:00,True,nan
4,212,3/6/18 17:44,0.03,BAR004,213,nan,1,9,77955197,0.09,4/28/21 13:54,True,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...
758,3476,4/19/21 12:00,nan,IN216,3145,nan,9,9,NaN,nan,4/19/21 12:05,True,nan
759,3477,4/19/21 12:00,nan,IN217,3146,nan,9,9,NaN,nan,4/19/21 12:05,True,nan
760,3478,4/28/21 14:07,0.00,CYOP21888,3147,nan,6,6,7.79833E+12,0.00,4/28/21 14:17,True,nan
761,3479,4/28/21 14:18,0.00,CYOP21871,3148,nan,6,6,1.77983E+13,0.00,4/28/21 14:18,True,nan


### product_template

In [21]:
product_template = pd.read_csv(folder+'product_template.csv')
product_template

,id,warranty,list_price,weight,sequence,color,write_uid,uom_id,description_purchase,default_code,create_date,create_uid,sale_ok,purchase_ok,message_last_post,company_id,uom_po_id,description_sale,description,volume,write_date,active,categ_id,name,rental,type,sale_line_warn,sale_line_warn_msg,track_service,invoice_policy,expense_policy,tracking,location_id,description_picking,warehouse_id,sale_delay,can_be_expensed,purchase_line_warn_msg,purchase_method,purchase_line_warn,use_time,life_time,removal_time,alert_time,produce_delay,landed_cost_ok,split_method,website_description,hs_code,tc_state,oc_state,secretaria_code,mercosur_code,product_brand_id,pack_cat,flavor_id,quote_description
0,209,nan,1.00,0.34,1,nan,8,20,nan,BAR002x12,3/6/18 17:44,1,True,False,nan,1,20,nan,nan,1.44,4/27/21 12:59,True,1,BARRA | Maní y cacao disp 12u x28g,False,product,no-message,nan,manual,order,no,lot,nan,nan,nan,0,False,NaN,receive,no-message,nan,180.00,nan,nan,0,NaN,equal,nan,NaN,NaN,NaN,nan,nan,1.00,1.00,36.00,NaN
1,210,nan,1.00,2.69,1,nan,9,22,nan,BAR002x12x8,3/6/18 17:44,1,True,False,nan,1,1,nan,nan,12.30,4/16/21 13:16,True,1,BARRA | Maní y cacao 8 disp 12u x28g (96u),False,product,no-message,nan,manual,order,no,lot,nan,nan,nan,0,False,NaN,receive,no-message,nan,180.00,nan,nan,0,NaN,equal,nan,NaN,NaN,NaN,nan,nan,1.00,1.00,36.00,NaN
2,211,nan,1.00,2.68,1,nan,6,41,nan,BAR004x3x32,3/6/18 17:44,1,False,False,nan,1,41,nan,nan,12.30,12/9/20 19:34,False,1,BARRA | Cajú y semillas de zapallo 32 tripack ...,False,product,no-message,nan,manual,order,no,lot,nan,nan,nan,0,False,NaN,receive,no-message,nan,180.00,nan,nan,0,NaN,equal,nan,NaN,NaN,NaN,nan,nan,nan,3.00,38.00,NaN
3,212,nan,1.00,0.08,1,nan,6,21,nan,BAR004x3,3/6/18 17:44,1,True,False,nan,1,21,nan,nan,0.36,1/20/21 18:00,True,1,BARRA | Cajú y semillas de zapallo tripack x28g,False,product,no-message,nan,manual,order,no,lot,nan,nan,nan,0,False,NaN,receive,no-message,nan,180.00,nan,nan,0,NaN,equal,nan,NaN,NaN,NaN,nan,nan,1.00,3.00,38.00,NaN
4,213,nan,1.00,0.03,1,nan,9,1,nan,BAR004,3/6/18 17:44,1,True,False,nan,1,1,nan,nan,0.09,4/28/21 13:54,True,1,BARRA | Cajú y semillas de zapallo x28g,False,product,no-message,nan,manual,order,no,lot,nan,nan,nan,0,False,NaN,receive,no-message,nan,180.00,nan,nan,0,NaN,equal,nan,NaN,NaN,NaN,nan,nan,1.00,2.00,38.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
758,3145,0.00,1.00,0.00,1,0.00,9,1,nan,IN216,4/19/21 12:00,9,False,True,nan,1,1,nan,nan,0.00,4/19/21 12:05,True,19,"Etiqueta Granola 260g - Cajú, coco y pasas",False,product,no-message,nan,manual,order,no,none,nan,nan,nan,0,False,NaN,receive,no-message,0.00,0.00,0.00,0.00,0,False,equal,nan,NaN,NaN,NaN,nan,nan,nan,nan,nan,NaN
759,3146,0.00,1.00,0.00,1,0.00,9,1,nan,IN217,4/19/21 12:00,9,False,True,nan,1,1,nan,nan,0.00,4/19/21 12:05,True,19,"Etiqueta Granola 260g - Nueces, manzana y pasas",False,product,no-message,nan,manual,order,no,none,nan,nan,nan,0,False,NaN,receive,no-message,0.00,0.00,0.00,0.00,0,False,equal,nan,NaN,NaN,NaN,nan,nan,nan,nan,nan,NaN
760,3147,0.00,0.00,0.00,1,0.00,6,54,nan,CYOP21888,4/28/21 14:07,6,True,True,nan,1,54,nan,nan,0.00,4/28/21 14:17,True,119,Bebida de Almendras Sin azúcar,False,service,no-message,nan,manual,order,no,none,nan,nan,nan,0,False,NaN,receive,no-message,0.00,0.00,0.00,0.00,0,False,equal,nan,NaN,NaN,NaN,nan,nan,11.00,15.00,75.00,NaN
761,3148,0.00,0.00,0.00,1,0.00,6,54,nan,CYOP21871,4/28/21 14:18,6,True,True,nan,1,54,nan,nan,0.00,4/28/21 14:25,True,119,Bebida de Almendras Original,False,consu,no-message,nan,manual,order,no,none,nan,nan,nan,0,False,NaN,receive,no-message,0.00,0.00,0.00,0.00,0,False,equal,nan,NaN,NaN,NaN,nan,nan,11.00,15.00,75.00,NaN


### product_uom

In [22]:
product_uom = pd.read_csv(folder+'product_uom.csv')
product_uom

,id,create_uid,name,rounding,write_uid,uom_type,write_date,factor,active,create_date,category_id,afip_code
0,1,1,Unit(s),0.01,1,reference,7/16/18 21:14,1.00,True,3/6/18 14:04,1,7.00
1,2,1,Dozen(s),0.01,1,bigger,7/16/18 21:14,0.08,True,3/6/18 14:04,1,9.00
2,3,1,kg,0.00,8,reference,1/24/19 15:51,1.00,True,3/6/18 14:04,2,1.00
3,5,1,Hour(s),0.01,1,smaller,3/6/18 14:08,8.00,True,3/6/18 14:04,3,98.00
4,6,1,Day(s),0.01,1,reference,3/6/18 14:08,1.00,True,3/6/18 14:04,3,98.00
5,9,1,km,0.01,1,bigger,3/6/18 14:08,0.00,True,3/6/18 14:04,4,17.00
6,10,1,cm,0.01,1,smaller,3/6/18 14:08,100.00,True,3/6/18 14:04,4,20.00
7,11,1,Liter(s),0.01,1,reference,3/6/18 14:08,1.00,True,3/6/18 14:04,5,5.00
8,14,1,inch(es),0.01,1,smaller,3/6/18 14:08,39.37,True,3/6/18 14:04,4,98.00
9,15,1,foot(ft),0.01,1,smaller,3/6/18 14:08,3.28,True,3/6/18 14:04,4,98.00


### res_company (prd2)

In [23]:
res_company = pd.read_csv(folder+'res_company.csv')
res_company

,id,name,partner_id,currency_id,rml_footer,create_date,rml_header,sequence,rml_paper_format,write_uid,logo_web,font,account_no,parent_id,email,create_uid,custom_footer,phone,rml_header2,rml_header3,write_date,rml_header1,company_registry,paperformat_id,fiscalyear_lock_date,bank_account_code_prefix,cash_account_code_prefix,anglo_saxon_accounting,fiscalyear_last_day,expects_chart_of_accounts,property_stock_valuation_account_id,property_stock_account_output_categ_id,transfer_account_id,currency_exchange_journal_id,period_lock_date,tax_calculation_rounding_method,accounts_code_digits,chart_template_id,overdue_msg,fiscalyear_last_month,property_stock_account_input_categ_id,days_between_two_followups,currency_interval_unit,currency_provider,currency_next_execution_date,sale_note,propagation_minimum_delta,internal_transit_location_id,po_lead,po_double_validation_amount,po_lock,po_double_validation,security_lead,localization,val2words_default,third_party_checks_bounced_endorsed_account_id,third_party_checks_cancelled_account_id,payment_method_validate_jr,own_check_rejected_account_id,holding_check_account_id,own_check_cancelled_account_id,deferred_check_account_id,rejected_check_account_id,check_deposit_offsetting_account,check_deposit_transfer_account_id,arg_sortdate,double_validation,afip_auth_verify_type,arba_cit,automatic_withholdings,manufacturing_lead,agip_password,agip_user,agip_enabler,analytic_tags_to_rows,min_days_between_followup,permanent_lock_date
0,1,Zafran,1,20,nan,nan,\n<header>\n <pageTemplate>\n <frame...,10,a4,13,binary data,nan,nan,nan,zafran@zafran.com.ar,nan,False,nan,"\n<header>\n<pageTemplate>\n <frame id=""fir...","\n<header>\n<pageTemplate>\n <frame id=""fir...",3/9/21 18:07,zafrán - Recetas honestas,nan,1,12/28/2019,111201,111101,False,31,True,nan,nan,478,4,12/28/2019,round_per_line,7,1,"Hola, \n\n\nAdjunto detalle de la cuenta corri...",12,nan,4,manually,ecb,nan,nan,0,17,0,5000,edit,one_step,0,argentina,1,577,578,False,580,480,579,79,576,bank_account,nan,True,True,not_available,nan,nan,0,nan,nan,nan,False,6,nan
1,1,Zafran,1,20,nan,nan,\n<header>\n <pageTemplate>\n <frame...,10,a4,13,binary data,nan,nan,nan,zafran@zafran.com.ar,nan,False,nan,"\n<header>\n<pageTemplate>\n <frame id=""fir...","\n<header>\n<pageTemplate>\n <frame id=""fir...",3/9/21 18:07,zafrán - Recetas honestas,nan,1,12/28/2019,111201,111101,False,31,True,nan,nan,478,4,12/28/2019,round_per_line,7,1,"Hola, \n\n\nAdjunto detalle de la cuenta corri...",12,nan,4,manually,ecb,nan,nan,0,17,0,5000,edit,one_step,0,argentina,1,577,578,False,580,480,579,79,576,bank_account,nan,True,True,not_available,nan,nan,0,nan,nan,nan,False,6,nan


### res_currency_rate (prd2)

In [24]:
res_currency_rate = pd.read_csv(folder+'res_currency_rate.csv')
res_currency_rate

,id,create_uid,name,company_id,write_uid,currency_id,rate,write_date,create_date,inverse_rate
0,1,6,1/10/2019 16:48,1,6,3,1.15,1/10/2019 16:48,1/10/2019 16:48,nan
1,2,6,1/10/2019 16:48,1,6,1,1.00,1/10/2019 16:48,1/10/2019 16:48,nan
2,3,1,4/22/2019 0:00,1,1,20,1.00,4/22/2019 13:52,4/22/2019 13:52,1.00
3,4,6,6/11/2019 16:22,1,6,3,0.02,6/11/2019 16:22,6/11/2019 16:22,45.70
4,5,6,6/11/2019 16:44,1,6,3,0.02,6/11/2019 16:44,6/11/2019 16:44,45.00
5,6,1,1/8/2019 3:02,1,1,3,0.03,6/21/2019 19:03,6/21/2019 19:02,36.46
6,7,1,5/2/2020 0:00,1,1,20,1.00,12/14/2020 19:16,12/14/2020 19:16,1.00
7,8,1,12/14/2020 19:17,1,1,20,1.00,12/14/2020 19:17,12/14/2020 19:17,1.00
8,1,6,1/10/2019 16:48,1,6,3,1.15,1/10/2019 16:48,1/10/2019 16:48,nan
9,2,6,1/10/2019 16:48,1,6,1,1.00,1/10/2019 16:48,1/10/2019 16:48,nan


### res_partner_market

In [25]:
res_partner_market = pd.read_csv(folder+'res_partner_market.csv')
res_partner_market

,id,create_uid,create_date,name,write_uid,write_date,categ_id
0,2,6,9/8/20 19:55,Dietética,6,9/14/20 15:06,1
1,3,6,9/8/20 19:57,Empresa,6,9/14/20 16:18,2
2,4,6,9/8/20 20:42,Distribuidor de Dietéticas,6,9/14/20 16:18,1
3,5,6,9/8/20 20:43,Cadena de dietética,6,9/14/20 16:18,1
4,6,6,9/8/20 20:43,Especial,6,9/14/20 16:18,5
5,7,6,9/8/20 20:43,Farmacia,6,9/14/20 16:19,4
6,8,6,9/8/20 20:43,Kiosco,6,9/14/20 16:19,4
7,9,6,9/8/20 20:43,On line,6,9/14/20 16:19,3
8,10,6,9/8/20 20:44,Otros,6,9/14/20 16:19,5
9,11,6,9/8/20 20:44,Petrolera,6,9/14/20 16:19,4


### res_partner

In [26]:
res_partner = pd.read_csv(folder+'res_partner.csv')
res_partner

,id,name,company_id,comment,website,create_date,color,active,street,supplier,city,display_name,zip,title,country_id,commercial_company_name,parent_id,company_name,employee,ref,email,is_company,function,lang,fax,street2,barcode,phone,write_date,date,tz,write_uid,customer,create_uid,credit_limit,user_id,mobile,type,partner_share,vat,state_id,commercial_partner_id,notify_email,message_last_post,opt_out,message_bounce,signup_type,signup_expiration,signup_token,team_id,debit_limit,last_time_entries_checked,invoice_warn_msg,invoice_warn,calendar_last_notif_ack,sale_warn_msg,sale_warn,picking_warn,picking_warn_msg,purchase_warn,purchase_warn_msg,main_id_number,main_id_category_id,gross_income_type,afip_responsability_type_id,gross_income_number,start_date,imp_ganancias_padron,actividad_monotributo_padron,integrante_soc_padron,last_update_padron,estado_padron,monotributo_padron,imp_iva_padron,empleador_padron,employee_id,drei,transport_id,contact_person,woo_customer_id,woo_company_name_ept,email_collections,apg_warn,apg_warn_msg,market_id,sale_quote_template_id,sale_discount
0,83459,Cliente 83459,1.00,nan,nan,12/22/20 19:24,0,True,Direccion cliente 83459,True,NaN,Cliente 83459,NaN,nan,nan,NaN,nan,nan,False,nan,nan,False,NaN,es_AR,nan,nan,nan,nan,3/4/21 15:31,nan,America/Argentina/Buenos_Aires,1,False,6.00,nan,nan,nan,contact,True,nan,nan,83459,always,nan,False,nan,NaN,nan,NaN,nan,nan,nan,NaN,no-message,nan,nan,no-message,no-message,nan,no-message,nan,nan,25.00,NaN,1.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,False,nan,nan,nan,nan,NaN,nan,nan,no-message,NaN,nan,nan,0
1,83992,Cliente 83992,1.00,nan,nan,3/4/21 15:50,0,True,Direccion cliente 83992,False,tigre,Cliente 83992,1648,nan,11.00,NaN,nan,nan,False,nan,nan,False,NaN,es_AR,nan,nan,nan,nan,3/4/21 15:50,nan,NaN,1,True,1.00,nan,nan,nan,NaN,True,nan,553.00,83992,always,nan,False,nan,NaN,nan,NaN,nan,nan,nan,NaN,no-message,nan,nan,no-message,no-message,nan,no-message,nan,nan,35.00,NaN,6.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,False,nan,nan,nan,nan,NaN,nan,nan,no-message,NaN,9.00,nan,0
2,7412,Cliente 7412,1.00,nan,nan,4/25/19 13:25,0,True,Direccion cliente 7412,True,NaN,Cliente 7412,NaN,nan,nan,NaN,nan,nan,False,nan,nan,False,NaN,es_AR,nan,nan,nan,nan,3/4/21 15:31,nan,America/Buenos_Aires,1,False,8.00,nan,nan,nan,contact,True,nan,nan,7412,always,nan,False,0.00,NaN,nan,NaN,nan,nan,nan,NaN,no-message,nan,nan,no-message,no-message,nan,no-message,nan,nan,25.00,NaN,7.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,False,nan,nan,nan,nan,NaN,nan,nan,no-message,NaN,nan,nan,0
3,10788,Cliente 10788,1.00,nan,nan,3/9/20 18:20,0,True,Direccion cliente 10788,False,CABA,Cliente 10788,1001,nan,11.00,Empresa 10788,nan,nan,False,nan,nan,True,NaN,es_AR,nan,nan,nan,nan,3/4/21 15:31,nan,NaN,1,True,1.00,nan,nan,nan,NaN,True,nan,552.00,10788,always,nan,False,nan,NaN,nan,NaN,nan,nan,nan,NaN,no-message,nan,nan,no-message,warning,nan,no-message,nan,nan,25.00,NaN,1.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,False,nan,nan,nan,nan,NaN,nan,nan,no-message,NaN,3.00,50.00,0
4,10789,Cliente 10789,1.00,nan,nan,3/9/20 18:20,0,False,Direccion cliente 10789,False,CABA,Cliente 10789,1001,nan,nan,Empresa 10789,10788.00,nan,False,nan,nan,False,NaN,es_AR,nan,nan,nan,nan,3/4/21 15:31,nan,NaN,1,False,1.00,0.00,nan,nan,delivery,True,nan,278.00,10788,always,nan,False,nan,NaN,nan,NaN,nan,0.00,nan,NaN,no-message,nan,nan,no-message,no-message,nan,no-message,nan,nan,25.00,NaN,6.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,False,nan,nan,nan,nan,NaN,nan,nan,no-message,NaN,nan,nan,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8592,84471,Cliente 84471,1.00,nan,nan,4/28/21 23:02,0,True,Direccion cliente 84471,False,Paso del Rey,Cliente 84471,1742,nan,11.00,NaN,nan,

### sale_order_line_invoice_rel

In [27]:
sale_order_line_invoice_rel = pd.read_csv(folder+'sale_order_line_invoice_rel.csv')
sale_order_line_invoice_rel

,invoice_line_id,order_line_id
0,1349,4723
1,1350,4724
2,1351,4725
3,1352,4726
4,1355,4712
...,...,...
53381,85486,309281
53382,85487,309282
53383,85488,309283
53384,85489,309284


### sale_order_line

In [28]:
sale_order_line = pd.read_csv(folder+'sale_order_line.csv')
sale_order_line

,id,create_date,product_uom,price_unit,product_uom_qty,price_subtotal,write_uid,currency_id,price_reduce_taxexcl,create_uid,price_tax,qty_to_invoice,customer_lead,layout_category_sequence,company_id,state,order_partner_id,order_id,qty_invoiced,sequence,discount,write_date,price_reduce,qty_delivered,layout_category_id,product_id,price_reduce_taxinc,price_total,invoice_status,name,salesman_id,product_packaging,route_id,is_delivery,woo_line_id,website_description
0,3620,9/7/18 19:22,22,1194.24,2.00,2316.83,6,20,1158.42,6,486.53,0,0,nan,1,sale,4849,220,2.00,10,3.00,10/8/18 23:42,1158.41,2.00,nan,246,1401.68,2803.36,invoiced,[MIX001x8x12] SNACK | Almendras y pasas 8 disp...,7,nan,nan,False,nan,NaN
1,3621,9/7/18 19:22,22,1194.24,2.00,2316.83,6,20,1158.42,6,486.53,0,0,nan,1,sale,4849,220,2.00,11,3.00,10/8/18 23:42,1158.41,2.00,nan,238,1401.68,2803.36,invoiced,"[MIX002x8x12] SNACK | Cajú, arándanos y curry ...",7,nan,nan,False,nan,NaN
2,3622,9/7/18 19:22,22,1194.24,2.00,2316.83,6,20,1158.42,6,486.53,0,0,nan,1,sale,4849,220,2.00,11,3.00,10/8/18 23:42,1158.41,2.00,nan,242,1401.68,2803.36,invoiced,[MIX004x8x12] SNACK | Frutos secos y salsa de ...,7,nan,nan,False,nan,NaN
3,3623,9/7/18 19:22,22,1194.24,2.00,2316.83,6,20,1158.42,6,486.53,0,0,nan,1,sale,4849,220,2.00,11,3.00,10/8/18 23:42,1158.41,2.00,nan,234,1401.68,2803.36,invoiced,[MIX008x8x12] SNACK | Almendras y maní picante...,7,nan,nan,False,nan,NaN
4,3624,9/7/18 19:22,22,1194.24,2.00,2316.83,6,20,1158.42,6,486.53,0,0,nan,1,sale,4849,220,2.00,11,3.00,10/8/18 23:42,1158.41,2.00,nan,217,1401.68,2803.36,invoiced,[BAR001x8x12] BARRA | Almendras y coco 8 disp ...,7,nan,nan,False,nan,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56696,309281,4/28/21 23:02,25,2654.54,1.00,2654.54,1,20,2654.54,1,557.45,0,0,nan,1,sale,84471,42869,1.00,10,0.00,4/28/21 23:02,2654.54,0.00,nan,221,3211.99,3211.99,invoiced,[BAR002x60] BARRA | Maní y cacao 60u x28g,1,nan,nan,False,32062.00,NaN
56697,309282,4/28/21 23:02,1,0.00,1.00,0.00,1,20,0.00,1,0.00,0,0,nan,1,sale,84471,42869,1.00,10,0.00,4/28/21 23:02,0.00,0.00,nan,3021,0.00,0.00,invoiced,Envío gratuito,1,nan,nan,True,32063.00,NaN
56698,309283,4/28/21 23:33,25,2654.54,1.00,2654.54,1,20,2654.54,1,557.45,0,0,nan,1,sale,74241,42870,1.00,10,0.00,4/28/21 23:33,2654.54,0.00,nan,3387,3211.99,3211.99,invoiced,[BAR006x60] BARRA | Arándanos y almendras 60u ...,1,nan,nan,False,32065.00,NaN
56699,309284,4/28/21 23:33,1,561.98,1.00,561.98,1,20,561.98,1,118.02,0,0,nan,1,sale,74241,42870,1.00,10,0.00,4/28/21 23:33,561.98,0.00,nan,271,680.00,680.00,invoiced,[GRA008] GRANOLA | Avellanas y semillas de zap...,1,nan,nan,False,32066.00,NaN


### stock_location (prd2)

In [29]:
stock_location = pd.read_csv(folder+'stock_location.csv')
stock_location

,id,parent_left,parent_right,comment,putaway_strategy_id,create_date,write_date,write_uid,partner_id,removal_strategy_id,scrap_location,location_id,company_id,complete_name,usage,create_uid,barcode,posz,posx,posy,active,name,return_location,valuation_in_account_id,valuation_out_account_id
0,9,1,2,NaN,nan,3/6/18 14:05,3/6/18 14:05,1,nan,nan,False,2.00,nan,Partner Locations/Customers,customer,1,nan,0,0,0,True,Customers,False,nan,nan
1,8,3,4,NaN,nan,3/6/18 14:05,3/6/18 14:05,1,nan,nan,False,2.00,nan,Partner Locations/Vendors,supplier,1,nan,0,0,0,True,Vendors,False,nan,nan
2,2,0,5,NaN,nan,3/6/18 14:05,3/6/18 14:05,1,nan,nan,False,nan,nan,Partner Locations,view,1,nan,1,0,0,True,Partner Locations,False,nan,nan
3,116,10,11,NaN,nan,1/15/21 20:31,1/15/21 20:31,9,nan,nan,False,114.00,1.00,Physical Locations/CER/Entrada,internal,9,nan,0,0,0,False,Entrada,False,nan,nan
4,117,8,9,NaN,nan,1/15/21 20:31,1/15/21 20:31,9,nan,nan,False,114.00,1.00,Physical Locations/CER/Control de calidad,internal,9,nan,0,0,0,False,Control de calidad,False,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,85,144,145,NaN,nan,8/29/19 19:30,8/29/19 19:35,1,nan,nan,False,83.00,1.00,Physical Locations/ISRAEL/Entrada,internal,6,nan,0,0,0,False,Entrada,False,nan,nan
140,83,141,154,NaN,nan,8/29/19 19:30,8/29/19 19:35,1,nan,nan,False,1.00,1.00,Physical Locations/ISRAEL,view,6,nan,0,0,0,True,ISRAEL,False,nan,nan
141,99,41,52,NaN,nan,7/3/20 13:07,7/3/20 13:07,9,nan,nan,False,1.00,1.00,Physical Locations/TGS,view,9,nan,0,0,0,True,TGS,False,nan,nan
142,115,16,17,NaN,nan,1/15/21 20:31,1/15/21 20:31,9,nan,nan,False,114.00,1.00,Physical Locations/CER/Zona de empaquetado,internal,9,nan,0,0,0,False,Zona de empaquetado,False,nan,nan


### stock_move (prd2)

In [30]:
stock_move = pd.read_csv(folder+'stock_move.csv')
stock_move

,id,origin,create_date,restrict_partner_id,product_uom,price_unit,product_uom_qty,procure_method,product_qty,partner_id,priority,picking_type_id,location_id,sequence,company_id,note,state,ordered_qty,origin_returned_move_id,product_packaging,restrict_lot_id,date_expected,procurement_id,create_uid,warehouse_id,inventory_id,partially_available,propagate,move_dest_id,date,scrapped,write_uid,product_id,push_rule_id,name,split_from,rule_id,location_dest_id,write_date,group_id,picking_id,purchase_line_id,to_refund_so,workorder_id,consume_unbuild_id,is_done,unit_factor,bom_line_id,raw_material_production_id,quantity_done_store,production_id,operation_id,unbuild_id,weight,weight_uom_id,subproduct_id
0,5,MO/00002,3/27/18 17:32,nan,1,nan,1.00,make_to_stock,1.00,nan,1,nan,7,10,1,nan,cancel,1.00,nan,nan,nan,3/27/18 17:32,nan,6,nan,nan,False,False,nan,3/27/18 17:32,False,1,267,nan,MO/00002,nan,nan,15,5/16/18 15:08,2.00,nan,nan,False,nan,nan,True,nan,nan,nan,nan,nan,nan,nan,1.00,3,nan
1,1,MO/00001,3/23/18 13:56,nan,1,nan,1.00,make_to_stock,1.00,nan,1,nan,7,10,1,nan,cancel,1.00,nan,nan,nan,3/23/18 13:56,nan,6,nan,nan,False,False,nan,3/23/18 13:56,False,1,252,nan,MO/00001,nan,nan,15,5/16/18 15:08,1.00,nan,nan,False,nan,nan,True,nan,nan,nan,nan,nan,nan,nan,3.00,3,nan
2,7,MO/00002,3/27/18 17:32,nan,1,0.00,1.00,make_to_stock,1.00,nan,1,nan,15,2,1,nan,cancel,1.00,nan,nan,nan,3/27/18 17:32,nan,6,1.00,nan,False,False,nan,3/27/18 17:32,False,1,675,nan,MO/00002,nan,nan,7,5/16/18 15:08,2.00,nan,nan,False,nan,nan,True,1.00,nan,nan,nan,nan,nan,nan,0.00,3,nan
3,8,MO/00002,3/27/18 17:32,nan,1,0.00,1.00,make_to_stock,1.00,nan,1,nan,15,2,1,nan,cancel,1.00,nan,nan,nan,3/27/18 17:32,nan,6,1.00,nan,False,False,nan,3/27/18 17:32,False,1,691,nan,MO/00002,nan,nan,7,5/16/18 15:08,2.00,nan,nan,False,nan,nan,True,1.00,nan,nan,nan,nan,nan,nan,0.00,3,nan
4,525,NaN,7/18/18 20:36,nan,1,nan,50.00,make_to_stock,50.00,nan,1,nan,15,10,1,nan,cancel,45.00,nan,nan,nan,7/19/18 0:00,nan,9,nan,nan,False,True,nan,7/18/18 0:00,False,9,2950,nan,/,nan,nan,34,7/18/18 20:55,nan,65.00,nan,False,nan,nan,True,nan,nan,nan,nan,nan,nan,nan,0.00,3,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295483,165915,SO0006236,3/9/21 20:39,nan,25,nan,1.00,make_to_order,1.00,4891.00,1,4.00,16,10,1,nan,assigned,1.00,nan,nan,nan,3/9/21 20:40,104878.00,1,1.00,nan,False,True,nan,3/9/21 20:07,False,7,3381,nan,[BAR005x60] BARRA | Chocolate y quinoa 60u x23g,nan,39.00,9,3/9/21 20:40,20678.00,41271.00,nan,False,nan,nan,False,nan,nan,nan,nan,nan,nan,nan,1.44,3,nan
295484,165916,SO0006236,3/9/21 20:39,nan,25,nan,1.00,make_to_order,1.00,4891.00,1,4.00,16,10,1,nan,assigned,1.00,nan,nan,nan,3/9/21 20:40,104879.00,1,1.00,nan,False,True,nan,3/9/21 20:07,False,7,3387,nan,[BAR006x60] BARRA | Arándanos y almendras 60u ...,nan,39.00,9,3/9/21 20:40,20678.00,41271.00,nan,False,nan,nan,False,nan,nan,nan,nan,nan,nan,nan,1.44,3,nan
295485,165917,SO0006236,3/9/21 20:39,nan,26,nan,1.00,make_to_order,1.00,4891.00,1,4.00,16,10,1,nan,assigned,1.00,nan,nan,nan,3/9/21 20:40,104880.00,1,1.00,nan,False,True,nan,3/9/21 20:07,False,7,257,nan,"[GAL002x20] GALLETITA | Avena, coco y lino 20u...",nan,39.00,9,3/9/21 20:40,20678.00,41271.00,nan,False,nan,nan,False,nan,nan,nan,nan,nan,nan,nan,3.00,3,nan
295486,165918,SO0006236,3/9/21 20:39,nan,26,nan,1.00,make_to_order,1.00,4891.00,1,4.00,16,10,1,nan,assigned,1.00,nan,nan,nan,3/9/21 20:40,104881.00,1,1.00,nan,False,True,nan,3/9/21 20:07,False,7,2934,nan,"[GAL003x20] GALLETITA | Algarroba, pasas y gir...",nan,39.00,9,3/9/21 20:40,20678.00,41271.00,nan,False,nan,nan,False,nan,nan,nan,nan,nan,nan,nan,3.00,3,nan


#### Duplicados

In [31]:
duplicates = stock_move[stock_move.duplicated()]
print('La cantidad de elementos duplicados es:', duplicates.shape[0])

La cantidad de elementos duplicados es: 147744


### stock_production_lot (prd2)

In [32]:
stock_production_lot = pd.read_csv(folder+'stock_production_lot.csv')
stock_production_lot

,id,create_date,write_uid,create_uid,message_last_post,ref,write_date,product_id,name,product_uom_id,life_date,removal_date,alert_date,use_date,origin,variety,brand
0,43,7/11/18 14:35,1,9,nan,NaN,11/15/18 17:14,448,28/2/19,3,NaN,2/28/19 20:20,NaN,NaN,nan,nan,NaN
1,288,7/31/18 19:12,9,9,nan,NaN,2/28/19 20:04,452,15/7/19,3,NaN,7/15/19 19:12,NaN,NaN,nan,nan,NaN
2,723,10/22/18 12:53,9,9,nan,NaN,2/28/19 20:04,452,5/9/2019,3,NaN,9/5/19 12:52,NaN,NaN,nan,nan,NaN
3,1093,12/20/18 15:52,8,8,nan,NaN,3/11/19 20:10,227,E 30/5/19,21,6/18/2019 15:51,5/30/19 3:00,NaN,NaN,nan,nan,NaN
4,240,7/27/18 20:01,1,9,nan,NaN,12/27/18 18:59,448,15/7/19,3,NaN,7/15/19 20:01,NaN,NaN,nan,nan,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11803,6242,3/2/21 19:30,1,9,nan,NaN,3/8/21 22:44,3204,26/8/21 Lote:058,26,8/26/2021 3:00,8/26/21 3:00,8/26/21 3:00,8/26/21 3:00,nan,nan,NaN
11804,6210,3/1/21 15:29,1,9,nan,NaN,3/5/21 13:04,274,8/8/21 L.04,29,NaN,8/11/21 3:00,NaN,NaN,nan,nan,NaN
11805,6130,2/19/21 18:15,1,9,nan,NaN,3/8/21 22:51,273,8/8/21 L.03,31,NaN,8/8/21 3:00,NaN,NaN,nan,nan,NaN
11806,6224,3/2/21 12:19,8,9,nan,ADD6280 Bavosi,3/9/21 13:31,461,2/11/2021,3,NaN,11/2/21 3:00,NaN,NaN,nan,nan,NaN


### stock_quant_move_rel (prd2)

In [33]:
stock_quant_move_rel = pd.read_csv(folder+'stock_quant_move_rel.csv')
stock_quant_move_rel

,move_id,quant_id
0,18,1
1,19,2
2,20,3
3,21,4
4,22,5
...,...,...
612593,155231,92556
612594,155232,92923
612595,155233,93142
612596,158070,96132


### stock_quant (prd2)

In [34]:
stock_quant = pd.read_csv(folder+'stock_quant.csv')
stock_quant

,id,create_date,qty,propagated_from_id,package_id,cost,lot_id,location_id,create_uid,reservation_id,company_id,owner_id,write_date,write_uid,product_id,packaging_type_id,negative_move_id,in_date,removal_date
0,183,7/16/18 19:27,17.40,nan,nan,45.00,2.00,7,1,nan,1,nan,7/23/18 13:23,1,422,nan,nan,7/11/18 14:41,7/10/19 18:31
1,557,7/31/18 16:20,18.00,nan,nan,50.00,259.00,7,1,nan,1,nan,7/31/18 16:26,1,424,nan,nan,7/27/18 20:47,7/20/19 20:40
2,586,7/31/18 16:25,51.00,nan,nan,0.00,270.00,7,1,nan,1,nan,7/31/18 16:41,1,2911,nan,nan,7/31/18 16:25,1/28/19 16:22
3,184,7/16/18 19:27,85.00,nan,nan,20.00,1.00,7,1,nan,1,nan,7/23/18 13:23,1,432,nan,nan,7/11/18 14:41,6/7/19 18:26
4,896,8/8/18 15:52,-12.00,nan,nan,0.00,nan,15,1,nan,1,nan,8/8/18 15:52,1,704,nan,1937.00,8/8/18 15:52,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192703,99110,3/8/21 12:53,31.00,nan,nan,1236.47,6278.00,82,1,165814.00,1,nan,3/9/21 14:58,1,3384,nan,nan,3/8/21 12:53,12/11/21 3:00
192704,99000,3/5/21 18:54,30.00,nan,nan,590.00,6181.00,7,1,nan,1,nan,3/9/21 20:09,1,428,nan,nan,2/25/21 14:10,7/6/22 3:00
192705,97935,3/1/21 15:20,5.68,nan,nan,579.90,6083.00,7,1,nan,1,nan,3/8/21 13:54,1,438,nan,nan,2/23/21 12:45,7/23/21 3:00
192706,96458,2/17/21 12:58,1.30,nan,nan,138.00,6086.00,7,1,nan,1,nan,3/8/21 13:54,1,451,nan,nan,2/17/21 12:58,2/28/22 3:00
